In [ ]:
! pip install transformers accelerate langchain bitsandbytes huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.9/260.9 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 61.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 82.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [ ]:
model="meta-llama/Llama-2-7b-chat-hf"

In [ ]:
tokenizer=AutoTokenizer.from_pretrained(model,use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:720: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
model=AutoModelForCausalLM.from_pretrained(model,
                                           device_map='auto',
                                           torch_dtype=torch.float16,
                                           use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:466: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
pipeline=transformers.pipeline("text-generation",
                               model=model,
                               tokenizer=tokenizer,
                               torch_dtype=torch.bfloat16,
                               device_map="auto",
                               # Maximum number of tokens to generate, a word is generally 2-3 tokens (minimum: 1)
                               max_new_tokens=512,
                               #Minimum number of tokens to generate. To disable this set it to -1. A word is generally 2-3 tokens (minimum 1)
                               min_new_tokens=-1,
                               #Adjusts the randomness of output
                               temperature=0.75,
                               do_sample=True,
                               top_k=30,
                               num_return_sequences=1,
                               eos_token_id=tokenizer.eos_token_id)

https://replicate.com/meta/llama-2-7b-chat


In [ ]:
llm=HuggingFacePipeline(pipeline=pipeline, model_kwargs={'temperature':0})

In [ ]:
llm.predict("Please provide a concise summary of the Book Harry Potter")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


" and the Philosopher's Stone. Here is a summary of the novel Harry Potter and the Philosopher's Stone by J.K. Rowling. Harry Potter is an orphan who lives with his cruel and neglectful.\nHarry Potter and the Philosopher's Stone is a novel by J.K. Rowling that follows the adventures of a young wizard named Harry Potter and his friends Ron Weasley and Hermione Granger. The story begins on Harry's eleventh birthday, when he receives a letter informing him that he has been accepted to attend Hogwarts School of Witchcraft and Wizardry. Harry learns that he is a wizard, and that his parents were murdered by the dark wizard Lord Voldemort.\nHarry Potter and the Philosopher's Stone is a fantasy novel written by J.K. Rowling. The story follows the adventures of a young wizard named Harry Potter, who attends Hogwarts School of Witchcraft and Wizardry. Harry discovers that he is famous in the wizarding world, and that his parents were murdered by the dark wizard Lord Voldemort. Harry learns that

In [ ]:
B_INST, E_INST= "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<<SYS>>\n\n"

In [ ]:
DEFAULT_SYSTEM_PROMPT="""\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
"""

In [18]:
instruction = "Convert the following text from English to French: \n\n{text}"

In [19]:
SYSTEM_PROMPT=B_SYS + DEFAULT_SYSTEM_PROMPT + E_SYS

In [20]:
SYSTEM_PROMPT

"<<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n\n<<SYS>>\n\n"

In [21]:
template = B_INST + SYSTEM_PROMPT + instruction + E_INST

In [22]:
template

"[INST]<<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n\n<<SYS>>\n\nConvert the following text from English to French: \n\n{text}[/INST]"

In [23]:
prompt=PromptTemplate(input_variables=["text"], template=template)

In [27]:
text ="How are you today"

In [28]:
LLM_Chain=LLMChain(llm=llm, prompt=prompt)

In [29]:
LLM_Chain.run(text)

'  Bonjour! Je suis heureux de vous aider. The English phrase "How are you today" can be translated to French as "Comment allez-vous aujourd\'hui."'

## Custom

In [30]:
instruction = "Convert the following text from English to French: \n\n{text}"

In [32]:
CUSTOM_SYSTEM_PROMPT="You are an advanced assistant that excels at translation."

In [33]:
SYSTEM_PROMPT=B_SYS+CUSTOM_SYSTEM_PROMPT+E_SYS

In [34]:
SYSTEM_PROMPT

'<<SYS>>\nYou are an advanced assistant that excels at translation.\n<<SYS>>\n\n'

In [35]:
template=B_INST+SYSTEM_PROMPT+instruction+E_INST

In [36]:
print(template)

[INST]<<SYS>>
You are an advanced assistant that excels at translation.
<<SYS>>

Convert the following text from English to French: 

{text}[/INST]


In [37]:
prompt=PromptTemplate(input_variables=["text"], template=template)

In [38]:
text ="How are you feeling today"

In [39]:
LLM_Chain=LLMChain(llm=llm, prompt=prompt)

In [40]:
print(LLM_Chain.run(text))

  Sure! Here's the translation of "How are you feeling today" from English to French:

Comment vas-tu aujourd'hui?
